## Project: **Système Intelligent de Gestion et d'Analyse de Mots de Passe (part 2)**

In [30]:
import re
import sqlite3
import bcrypt
import random
import string
import pandas as pd
import numpy as np
import spacy, joblib
# from ipynb.fs.full.train import test_password

path = "C:\\Users\\User\\PycharmProjects\\PassPy\\training\\"

### 3. Génération de Mots de Passe sécurisés

In [31]:
import spacy, string, random
# Load English and French models
nlp_en = spacy.load("en_core_web_sm")
nlp_fr = spacy.load("fr_core_news_sm")

def resembles_common_word(password):
    # Check against both English and French models
    for nlp in [nlp_en, nlp_fr]:
        doc = nlp(password)
        for token in doc:
            if token.is_alpha and token.text.lower() in nlp.vocab:
                return True
    return False

def load_easy_patterns(file_path="easy_patterns.txt"):
    # Load patterns from the specified file
    with open(file_path, "r") as f:
        patterns = [line.strip() for line in f.readlines()]
    return patterns

def has_easy_patterns(password, patterns):
    # Check for sequential characters
    for i in range(len(password) - 2):
        if (ord(password[i+1]) == ord(password[i]) + 1) and (ord(password[i+2]) == ord(password[i]) + 2):
            return True
        if password[i] == password[i+1] == password[i+2]:  # Repeated characters
            return True
    # Check for patterns in the list
    for pattern in patterns:
        if pattern in password:
            return True
    return False

In [32]:
def generate_secure_password(length=12):
    if length < 8:
        raise ValueError("Password length must be at least 8 characters.")

    all_chars = string.ascii_letters + string.digits + string.punctuation
    easy_patterns = load_easy_patterns("easy_patterns.txt")
    
    while True:
        password = ''.join(random.choice(all_chars) for _ in range(length))
        if not has_easy_patterns(password, easy_patterns) and not resembles_common_word(password):
            return password


In [6]:
secure_password = generate_secure_password(12)
print("Generated secure password:", secure_password)

Generated secure password: !i\?M"h{3Zd4


### 4. Stockage sécurisé de Mot de Passe

##### 4.1. Encryption imp

In [33]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os
import base64

# Generate a key for AES encryption
def generate_key(master_password, salt):
    kdf = PBKDF2HMAC(
        algorithm=SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(master_password.encode())

# Encrypt plaintext
def encrypt(plaintext, key):
    # Add padding
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(plaintext.encode()) + padder.finalize()

    # AES encryption
    iv = os.urandom(16)  # Generate a random IV
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    # Return IV and ciphertext
    return base64.b64encode(iv + ciphertext).decode()

# Decrypt ciphertext
def decrypt(ciphertext, key):
    data = base64.b64decode(ciphertext)

    # Extract IV and encrypted data
    iv = data[:16]
    encrypted_data = data[16:]

    # AES decryption
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Remove padding
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(padded_data) + unpadder.finalize()

    return plaintext.decode()


In [34]:
import bcrypt

# Hash the master password
def hash_password(password):
    salt = bcrypt.gensalt()
    hashed = bcrypt.hashpw(password.encode(), salt)
    return hashed

# Verify the master password
def verify_password(password, hashed):
    return bcrypt.checkpw(password.encode(), hashed)

##### 4.2. Store passwords in a Database

In [35]:
# Database setup
def initialize_database():
    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()

    # Users table
    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      username TEXT NOT NULL,
                      password TEXT NOT NULL)''')
    # Password table 
    cursor.execute('''CREATE TABLE IF NOT EXISTS passwords (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      account TEXT NOT NULL,
                      username TEXT,
                      password_hash TEXT NOT NULL,
                      password_encr TEXT NOT NULL,
                      user_id INTEGER,
                      FOREIGN KEY(user_id) REFERENCES users(id) )''')
    conn.commit()
    conn.close()

# Store password securely
def store_password(account, username, password, user_id):
    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()

    # Hash the password
    # password_hash = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
    password_hash = hash_password(password)

    # Generate AES key and crypt the pass
    master_password = "my0;ma5teR_Pa5sW0rd"
    salt = b'secure_salt'  # Use a secure and consistent salt
    key = generate_key(master_password, salt)
    # Encrypt the password
    password_encr = encrypt(password, key)

    # Insert into database
    cursor.execute('INSERT INTO passwords (account, username, password_hash, password_encr, user_id) VALUES (?, ?, ?, ?, ?)',
                   (account, username, password_hash, password_encr, user_id))
    conn.commit()
    conn.close()

# Retrieve passwords
def retrieve_passwords(user_id):
    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()

    # Retrieve all passwords for the logged-in user
    # cursor.execute('SELECT id, account, username FROM passwords WHERE user_id = ?', (user_id,))
    cursor.execute('SELECT * FROM passwords WHERE user_id = ?', (user_id,))
    passwords = cursor.fetchall()
    conn.close()
    return passwords

# Retrieve specific password
def retrieve_apass(user_id, account):
    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()

    # Retrieve a specific password record for the user based on the account name
    cursor.execute('SELECT id, account, username, password_encr FROM passwords WHERE user_id = ? AND account = ?', 
                   (user_id, account))
    password_record = cursor.fetchone()
    conn.close()

    if not password_record:
        return None

    # Decrypt the password
    master_password = "my0;ma5teR_Pa5sW0rd"
    salt = b'secure_salt'
    key = generate_key(master_password, salt)
    decrypted_password = decrypt(password_record[3], key)

    return {
        "id": password_record[0],
        "account": password_record[1],
        "username": password_record[2],
        "password": decrypted_password
    }

# Update an account password
def update_pass(user_id, account, new_username, new_password):
    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()

    # Hash the new password
    password_hash = hash_password(new_password)

    # Encrypt the new password
    master_password = "my0;ma5teR_Pa5sW0rd"
    salt = b'secure_salt'
    key = generate_key(master_password, salt)
    password_encr = encrypt(new_password, key)

    # Update the password record in the database
    cursor.execute('''
        UPDATE passwords 
        SET username = ?, password_hash = ?, password_encr = ? 
        WHERE user_id = ? AND account = ?''', 
        (new_username, password_hash, password_encr, user_id, account))
    conn.commit()
    conn.close()
    return cursor.rowcount > 0  # Return True if a row was updated

# Delete password
def delete_pass(user_id, account):
    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()

    # Delete the password record for the given account and user
    cursor.execute('DELETE FROM passwords WHERE user_id = ? AND account = ?', (user_id, account))
    conn.commit()
    conn.close()
    return cursor.rowcount > 0  # Return True if a row was deleted



##### 4.3. Save and Get the password

In [15]:
def save_password(master_password, account, username, password):
    # Generate AES key
    salt = b'secure_salt'  # Use a secure and consistent salt
    key = generate_key(master_password, salt)
    
    # Encrypt the password
    encrypted_password = encrypt(password, key)
    
    # Store in the database
    store_password(account, username, encrypted_password)

def get_password(master_password, account):
    # Retrieve encrypted data
    result = retrieve_password(account)
    if result is None:
        return None

    username, encrypted_password = result

    # Generate AES key
    salt = b'secure_salt'  # Use the same salt
    key = generate_key(master_password, salt)

    # Decrypt the password
    password = decrypt(encrypted_password, key)
    return username, password

In [11]:
master_password = "my_master_password"
ent_pass = "se]{13$`~cu@ssw0rd"

salt = b'to-90o^^?'  # Use a secure and consistent salt
key = generate_key(master_password, salt)
    
# Encrypt the password
encrypted_password = encrypt(ent_pass, key)

print(encrypted_password, len(encrypted_password))


# Generate AES key
salt = b'to-90o^^?'  # Use the same salt
key = generate_key(master_password, salt)

# Decrypt the password
out_pass = decrypt(encrypted_password, key)
print(out_pass)
hashed = hash_password(ent_pass)
print(hashed)


hN3WDcIUBV8rg+gtqRBZKdtm0a9vlh2tB1o8xHYnCcaLW70/vaTZC9IMdPxHYWhu 64
se]{13$`~cu@ssw0rd
b'$2b$12$P/xCgdWBPLRgoMWyiGNquODhSUa2xQr1yN/x1.t8t0oGTj/UzyjPy'


In [39]:
verify_password(ent_pass, b'$2b$12$xgcmmjYtxddCxFrncsmTD.SIu1hT4NY.RoYvEglul36C5aGdNYqjK')

True

In [ ]:
# Save a password
initialize_db()
master_password = "my_master_password"

save_password(master_password, "example.com", "user@example.com", "secureP@ssw0rd")
print("Password saved securely!")

# ------------------------ Retrieve a Password ----------------------
username, password = get_password("my_master_password", "example.com")
if username and password:
    print(f"Username: {username}, Password: {password}")
else:
    print("Account not found or incorrect master password.")
b'$2b$12$eYcS5oc5A.NCR4UBCTqq4esanMcW7bddxT2WfuGx1V8sd8Ibkqxnq'
b'$2b$12$h96MSwKpIdyB92HfVSLXY.b/f57cSptfo5AQrInhCFFhQYclip6OO'
b'$2b$12$V9crHP.S/J.ahmtavrsFSelgmjcBBLLxyKowwdON3ysGCCZbebhba'
b'$2b$12$xgcmmjYtxddCxFrncsmTD.SIu1hT4NY.RoYvEglul36C5aGdNYqjK'

### 5. Simulation d'attaques

In [36]:
import itertools
import hashlib
import bcrypt
import string
import time
from typing import Tuple

# Detect hash type based on length and pattern
def detect_hash_type(hashed_password: str) -> str:
    if hashed_password.startswith("$2b$") or hashed_password.startswith("$2a$"):
        return "bcrypt"
    elif len(hashed_password) == 32:
        return "md5"
    elif len(hashed_password) == 40:
        return "sha1"
    elif len(hashed_password) == 64:
        return "sha256"
    else:
        return "unknown"

# Compare a hashed password with its plaintext version
def compare_hash(plaintext: str, hashed_password: str, hash_type: str) -> bool:
    if hash_type == "bcrypt":
        return bcrypt.checkpw(plaintext.encode(), hashed_password.encode())
    elif hash_type == "md5":
        return hashlib.md5(plaintext.encode()).hexdigest() == hashed_password
    elif hash_type == "sha1":
        return hashlib.sha1(plaintext.encode()).hexdigest() == hashed_password
    elif hash_type == "sha256":
        return hashlib.sha256(plaintext.encode()).hexdigest() == hashed_password
    else:
        return False

##### 5.1. Attaques par Brute-force

In [37]:
# Brute-force attack
def brute_force_attack(hashed_password: str, hash_type: str, max_length: int = 6) -> Tuple[str, float]:
    characters = string.ascii_letters + string.digits + string.punctuation
    start_time = time.time()

    for length in range(1, max_length + 1):
        for combination in itertools.product(characters, repeat=length):
            candidate = ''.join(combination)
            if compare_hash(candidate, hashed_password, hash_type):
                elapsed_time = time.time() - start_time
                return candidate, elapsed_time

    elapsed_time = time.time() - start_time
    return None, elapsed_time

In [46]:
# Test in the test section
plaintext = "abc"
print(hashlib.md5(plaintext.encode()).hexdigest())
print(hash_password(plaintext).decode("utf-8"))

900150983cd24fb0d6963f7d28e17f72
$2b$12$a23oBx1yVi0hEiudWLyil.Art6XI7i4aR03kjOh914kos5V.R8Wpy


###### Last Tries (hashlib.sha256 and bcrypt)

In [43]:
# Brute-force attack engine
def brute_force_attack(hash_to_crack, charset, max_length):
    start_time = time.time()
    for length in range(1, max_length + 1):
        # Generate all combinations of the given charset
        for combination in itertools.product(charset, repeat=length):
            password = ''.join(combination)
            hashed_password = hashlib.sha256(password.encode()).hexdigest()
            
            if hashed_password == hash_to_crack:
                elapsed_time = time.time() - start_time
                return password, elapsed_time
    return None, time.time() - start_time

In [44]:
# Example hash to crack (hashed version of "abc")
target_password = "abc"
hashed_target = hashlib.sha256(target_password.encode()).hexdigest()

# Define character set and maximum password length
# charset = "0123456789" + "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKMNOPQRSTUVWYZ" + "<><>.,/?€~;':[]`!@#$%^&*()+-_=\"|\\"
charset = string.ascii_letters + string.digits + string.punctuation
max_length = 8

# Run brute-force attack
cracked_password, time_taken = brute_force_attack(hashed_target, charset, max_length)
if cracked_password:
    print(f"Password cracked: {cracked_password} (Time taken: {time_taken:.2f} seconds)")
else:
    print(f"Password not found (Time taken: {time_taken:.2f} seconds)")

Password cracked: abc (Time taken: 0.04 seconds)


In [47]:
import bcrypt
import itertools
import string

def brute_force_password(hashed_password, max_length=3):
    """
    Attempt to brute-force a password hashed with bcrypt.
    
    Args:
        hashed_password (bytes): The bcrypt hash of the password to guess.
        max_length (int): The maximum length of the password to guess.
    
    Returns:
        str: The guessed password if successful, or None if not.
    """
    chars = string.ascii_letters + string.digits + string.punctuation  # Characters to try
    print("Starting brute-force attack...")
    
    # Generate all possible passwords up to max_length
    for length in range(1, max_length + 1):
        for attempt in itertools.product(chars, repeat=length):
            candidate = ''.join(attempt)
            
            if bcrypt.checkpw(candidate.encode(), hashed_password):
                print(f"Password found: {candidate}")
                return candidate
    
    print("Password not found within given constraints.")
    return None

In [51]:
password = "7"
hashed = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
print(f"Hashed Password: {hashed.decode()}")

# Brute-force attempt (educational)
brute_force_password(hashed, max_length=3)

Hashed Password: $2b$12$BGeZy5XTCT/NXtRe8HaCc./.qvngxoPhFFHwz8j7AdX3R5fukX3KS
Starting brute-force attack...
Password found: 7


'7'

##### 5.2. Attaques par dictionnaire

In [38]:
def dictionary_attack(hashed_password: str, hash_type: str, dictionary_file: str = "dictionary.txt") -> Tuple[str, float]:
    start_time = time.time()

    try:
        with open(dictionary_file, "r") as file:
            for line in file:
                candidate = line.strip()
                if compare_hash(candidate, hashed_password, hash_type):
                    elapsed_time = time.time() - start_time
                    return candidate, elapsed_time
    except FileNotFoundError:
        print("Dictionary file not found!")

    elapsed_time = time.time() - start_time
    return None, elapsed_time


In [ ]:
# Test (in the test section)


###### Avec hashlib

In [ ]:
def dictionary_attack(hash_to_crack, dictionary_file):
    start_time = time.time()
    with open(dictionary_file, 'r') as file:
        for line in file:
            password = line.strip()
            hashed_password = hashlib.sha256(password.encode()).hexdigest()
            
            if hashed_password == hash_to_crack:
                elapsed_time = time.time() - start_time
                return password, elapsed_time
    return None, time.time() - start_time

In [ ]:
dictionary_file = "dictionary.txt"

# Run dictionary attack
cracked_password, time_taken = dictionary_attack(hashed_target, dictionary_file)
if cracked_password:
    print(f"Password cracked: {cracked_password} (Time taken: {time_taken:.2f} seconds)")
else:
    print(f"Password not found (Time taken: {time_taken:.2f} seconds)")

###### Avec bcript

In [ ]:
def dictionary_attack(target_hash, dictionary_file):
    """
    Perform a dictionary attack to guess a password from a hash.

    Args:
        target_hash (bytes): The bcrypt hash of the password to guess.
        dictionary_file (str): Path to the file containing potential passwords.

    Returns:
        str: The guessed password if found, or None if not.
    """
    try:
        with open(dictionary_file, "r") as file:
            for line in file:
                password = line.strip()  # Remove any trailing spaces or newlines
                
                # Hash the password and compare it with the target hash
                if bcrypt.checkpw(password.encode(), target_hash):
                    return password  # Password found
    except FileNotFoundError:
        print(f"Dictionary file {dictionary_file} not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return None  # Password not found


In [ ]:
# The target hash to crack (hashed with bcrypt)
password = "mypassword"
hashed_password = bcrypt.hashpw(password.encode(), bcrypt.gensalt())

# Path to the dictionary file
dictionary_path = "common_passwords.txt"

# Attempt dictionary attack
print("Attempting dictionary attack...")
guessed_password = dictionary_attack(hashed_password, dictionary_path)

if guessed_password:
    print(f"Password cracked! The password is: {guessed_password}")
else:
    print("Password could not be guessed.")

### 6. Tests et Validation

In [16]:
print("++++++++++++++++++++++++++++++++++++ Test and Validation +++++++++++++++++++++++++++++++")
print("<<<<<----------  Ok! --------------")

++++++++++++++++++++++++++++++++++++ Test and Validation +++++++++++++++++++++++++++++++
<<<<<----------  Ok! --------------


In [42]:
import sqlite3
import bcrypt
import os
from getpass import getpass
from cryptography.fernet import Fernet
import random
import string
import joblib 

# Initialize the database
def initialize_database():
    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()

    # Create users table
    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      username TEXT NOT NULL UNIQUE,
                      password TEXT NOT NULL)''')

    # Create passwords table
    cursor.execute('''CREATE TABLE IF NOT EXISTS passwords (
                      id INTEGER PRIMARY KEY AUTOINCREMENT,
                      account TEXT NOT NULL,
                      username TEXT,
                      password_hash TEXT NOT NULL,
                      password_encr TEXT NOT NULL,
                      user_id INTEGER,
                      FOREIGN KEY(user_id) REFERENCES users(id))''')
    conn.commit()
    conn.close()

# Register a new user
def register_user():
    username = input("Enter a username: ")
    password = getpass("Enter a password: ")
    hashed_password = bcrypt.hashpw(password.encode(), bcrypt.gensalt())

    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()
    try:
        cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, hashed_password))
        conn.commit()
        print("User registered successfully!")
    except sqlite3.IntegrityError:
        print("Username already exists!")
    conn.close()

# Authenticate user
def authenticate_user():
    username = input("Enter your username: ")
    password = getpass("Enter your password: ")

    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()
    cursor.execute('SELECT id, password FROM users WHERE username = ?', (username,))
    user = cursor.fetchone()
    conn.close()

    if user and bcrypt.checkpw(password.encode(), user[1]):
        print("Login successful!")
        return user[0]
    else:
        print("Invalid username or password!")
        return None

# Add a new password record
def add_password(user_id):
    account = input("Enter account name: ")
    username = input("Enter username: ")
    password = getpass("Enter password: ")
    
    store_password(account, username, password, user_id)

    print("Password added successfully!")

# View passwords for the logged-in user
def view_passwords(user_id):
    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()
    cursor.execute('SELECT id, account, username, password_encr, password_hash FROM passwords WHERE user_id = ?', (user_id,))
    passwords = cursor.fetchall()
    conn.close()

    # master_password = "mymasterpassword"
    # salt = b'secure_salt'
    # key = generate_key(master_password, salt)

    print("\nYour saved passwords:")
    print("{:<5} {:<30} {:<20} {:<64} {:<60}".format("ID", "Account", "Username", "Encrypted Password", "Hashed Password"))
    for row in passwords:
        # decrypted_password = decrypt(row[3], key)
        print("{:<5} {:<30} {:<20} {:<64}".format(row[0], row[1], row[2], row[3]), end=" ")
        print(row[4] )

# Update a password record
def update_password(user_id):
    view_passwords(user_id)
    record_id = int(input("Enter the ID of the record you want to update: "))
    new_username = input("Enter new username (leave blank to keep unchanged): ")
    new_password = getpass("Enter new password (leave blank to keep unchanged): ")

    conn = sqlite3.connect('password_manager.db')
    cursor = conn.cursor()

    if new_username:
        cursor.execute('UPDATE passwords SET username = ? WHERE id = ? AND user_id = ?', (new_username, record_id, user_id))

    if new_password:
        # Hash the new password
        password_hash = hash_password(new_password)
    
        # Encrypt the new password
        master_password = "my0;ma5teR_Pa5sW0rd"
        salt = b'secure_salt'
        key = generate_key(master_password, salt)
        password_encr = encrypt(new_password, key)
        cursor.execute('UPDATE passwords SET password_hash = ?, password_encr = ? WHERE id = ? AND user_id = ?',
                       (password_hash, password_encr, record_id, user_id))

    conn.commit()
    conn.close()
    print("Password record updated successfully!")

# Dictionary and brute-force attacks (basic simulation)
def password_attacks():
    
    print("Welcome to Password Attack Simulator")
    hashed_password = input("Enter the hashed password: ")

    # Detect hash type
    hash_type = detect_hash_type(hashed_password)
    if hash_type == "unknown":
        print("Error: Unsupported or unrecognized hash type.")
        return

    print(f"Detected hash type: {hash_type}")

    print("\nChoose an attack method:")
    print("1. Brute-Force Attack")
    print("2. Dictionary Attack")
    choice = input("Enter your choice: ")

    if choice == "1":
        max_length = int(input("Enter the maximum length for brute-force (e.g., 6): "))
        print("Performing brute-force attack...")
        result, time_taken = brute_force_attack(hashed_password, hash_type, max_length)
    elif choice == "2":
        dictionary_file = input("Enter the dictionary file path (e.g., passwords.txt): ")
        print("Performing dictionary attack...")
        result, time_taken = dictionary_attack(hashed_password, hash_type, dictionary_file)
    else:
        print("Invalid choice!")
        return

    if result:
        print(f"Password cracked: {result}")
    else:
        print("Failed to crack the password.")
    print(f"Time taken: {time_taken:.2f} seconds")

# Generate a strong password
def generate_strong_password():
    length = int(input("Enter the desired length of the password (min 8): "))
    if length < 8:
        print("Password length must be at least 8 characters.")
        return None
    secure_password = generate_secure_password(length)
    print("Generated password:", secure_password)
    

# Test the strength of a password
def test_password_strength():
    password = input("Enter the password to test: ")
    
    strength = test_password(password)
    print(f"The password strength is: {strength}")


#---------------------- Feature extraction functions -----------------------------------
def count_uppercase(password):
    return sum(1 for c in password if c.isupper())

def count_lowercase(password):
    return sum(1 for c in password if c.islower())

def count_digits(password):
    return sum(1 for c in password if c.isdigit())

def count_special_chars(password):
    return sum(1 for c in password if not c.isalnum())

def has_sequential_chars(password):
    # Check for sequences of 3 or more characters
    sequential = False
    for i in range(len(password) - 2):
        substr = password[i:i+3]
        if substr.isalpha() or substr.isdigit():
            if ord(substr[1]) == ord(substr[0]) + 1 and ord(substr[2]) == ord(substr[1]) + 1:
                sequential = True
                break
    return int(sequential)

def calculate_entropy(password):
    # Calculate Shannon entropy
    entropy = 0
    length = len(password)
    if length == 0:
        return 0
    chars = set(password)
    for c in chars:
        p = password.count(c) / length
        entropy -= p * np.log2(p)
    return entropy
#----------------------------------------------------------------------------
# Load the saved model and label encoder
loaded_model = joblib.load('random_forest_model.pkl') # random_forest_model.pkl | mlp_model.pkl | knn_model.pkl
loaded_label_encoder = joblib.load('label_encoder.pkl')

def test_password(password):    
    # Extract features
    features = {
        'length': len(password),
        'uppercase': count_uppercase(password),
        'lowercase': count_lowercase(password),
        'digits': count_digits(password),
        'special_chars': count_special_chars(password),
        'sequential': has_sequential_chars(password),
        'entropy': calculate_entropy(password)
    }

    # Convert to a DataFrame with proper feature names
    input_features = pd.DataFrame([features])

    # Predict strength
    predicted_strength = loaded_model.predict(input_features)
    decoded_strength = loaded_label_encoder.inverse_transform(predicted_strength)

    return decoded_strength[0]

# Main application logic
def main():
    initialize_database()
    print("Welcome to the Password Manager CLI App!")

    while True:
        print("\nChoose an option:")
        print("1. Login")
        print("2. Register")
        print("3. Exit")
        choice = input("Enter choice: ")

        if choice == "1":
            user_id = authenticate_user()
            if user_id:
                while True:
                    print("\nDashboard")
                    print("1. View Passwords")
                    print("2. Add Password")
                    print("3. Update Password")
                    print("4. Generate strong pasword")
                    print("5. Password strength")
                    print("6. Perform Password Attack")
                    print("7. Logout")
                    user_choice = input("Enter choice: ")

                    if user_choice == "1":
                        view_passwords(user_id)
                    elif user_choice == "2":
                        add_password(user_id)
                    elif user_choice == "3":
                        update_password(user_id)
                    elif user_choice == "4":
                        generate_strong_password()
                    elif user_choice == "5":
                        test_password_strength()
                    elif user_choice == "6":
                        password_attacks()
                    elif user_choice == "7":
                        break
                    else:
                        print("Invalid choice!")
        elif choice == "2":
            register_user()
        elif choice == "3":
            print("Goodbye!")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()


Welcome to the Password Manager CLI App!

Choose an option:
1. Login
2. Register
3. Exit


Enter choice:  1
Enter your username:  user1
Enter your password:  ········


Login successful!

Dashboard
1. View Passwords
2. Add Password
3. Update Password
4. Generate strong pasword
5. Password strength
6. Perform Password Attack
7. Logout


Enter choice:  1



Your saved passwords:
ID    Account                        Username             Encrypted Password                                               Hashed Password                                             
1     trello.com                     mytrello             UKVXKr9/71SNFrcarGMeWau4YjzsGPFFq3mUNyr+ce0=                     b'$2b$12$2j1oV79cxs12ozRqD98lreBqbjSZY/vvEIwQ49pQihzMWGL75YdLC'

Dashboard
1. View Passwords
2. Add Password
3. Update Password
4. Generate strong pasword
5. Password strength
6. Perform Password Attack
7. Logout


Enter choice:  7



Choose an option:
1. Login
2. Register
3. Exit


Enter choice:  3


Goodbye!


In [42]:
print(len("5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8"))
print(len("$2a$10$N9qo8uLOickgx2ZMRZoMyeIjZAgcfl7p92ldGxad68LJZdL17lhWy"))

print(len("hN3WDcIUBV8rg+gtqRBZKdtm0a9vlh2tB1o8xHYnCcaLW70/vaTZC9IMdPxHYWhu"))

64
60
64


In [41]:
print("<<<<<----------  Ok! --------------")

<<<<<----------  Ok! --------------
